In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import os, sys, time
sys.path.append("..")
from all_funcs import util
from model import Generator, Discriminator, train_discriminator, train_generator
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from sklearn.impute import SimpleImputer
tf.keras.backend.set_floatx('float64')
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)

/home/cheyu/anaconda3/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# from numba import cuda
# os.environ['CUDA_VISIBLE_DEVICES']="0"
# print(tf.test.is_gpu_available())

In [3]:
df=pd.read_csv("../dataset/df_noOutliner_ana.csv",index_col=0)
df, imp_mode, imp_mean=util.FeatureArrange(df)

In [4]:
## reduce redundant features which can be assembled
dataset=df.drop(['NIHTotal','THD_ID','cortical_CT', 'subcortical_CT',
              'circulation_CT', 'CT_find', 'watershed_CT', 'Hemorrhagic_infarct_CT',
              'CT_left', 'CT_right',],axis=1)

In [5]:
sc = MinMaxScaler()
dataset.loc[:,dataset.columns!='elapsed_class'] = sc.fit_transform(dataset.loc[:,dataset.columns!='elapsed_class'])


In [6]:
## setting hyperparameter
latent_dim = dataset.shape[1]
epochs = 10000
batch_size= 128
buffer_size = 6000
# save_interval = 50
n_critic = 5
checkpoint_dir = './training_checkpoints'


In [7]:
generator = Generator()
discriminator = Discriminator()

In [8]:
## create Cross Entropy
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [9]:
gen_opt = tf.keras.optimizers.Adam(0.0001,)
disc_opt = tf.keras.optimizers.Adam(0.0001,)

In [10]:
# save checkpoints
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=gen_opt,
                                 discriminator_optimizer=disc_opt,
                                 generator=generator,
                                 discriminator=discriminator)

In [11]:
# separate to 0,1 dataset
data_1=dataset.loc[dataset['elapsed_class']==1]
data_0=dataset.loc[dataset['elapsed_class']==0]

In [12]:
for epoch in range(epochs):
    start = time.time()
    disc_loss = 0
    gen_loss = 0

    # resample the dataset
    data1_shape_0 = data_1.sample(data_0.shape[0])
    df_same_shape = pd.concat([data1_shape_0, data_0])

    # slices to data and labels
    df_same_ = df_same_shape.iloc[:, :-1].to_numpy()
    org_label = df_same_shape.iloc[:, -1].to_numpy().reshape(-1, 1)

    # create batch dataset
    training_dataset = tf.data.Dataset.from_tensor_slices((df_same_, org_label))\
        .shuffle(buffer_size).batch(batch_size, drop_remainder=True)

    for data, labels in training_dataset:

        disc_loss += train_discriminator(data, generator,
                                         discriminator, disc_opt, latent_dim)

        if disc_opt.iterations.numpy() % n_critic == 0:
            gen_loss+= train_generator(data, generator,
                                        discriminator, gen_opt, batch_size, latent_dim)
    print('Time for epoch {} is {} sec - gen_loss = {}, disc_loss = {}'.format(epoch + 1, time.time() - start,
                                                                               gen_loss / batch_size,
                                                                               disc_loss / (batch_size*n_critic)))


If you intended to run this layer in float64, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float32 by default, call `tf.keras.backend.set_floatx('float32')`. To change just this layer, pass dtype='float32' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf_result: 0.32850263572607236
weight_L2 10.084361991645
tf_result: 0.080484837068006854
weight_L2 20.16469281625006
tf_result: 0.31112915481184922
weight_L2 10.520196834751657
tf_result: -0
tf_result: 0.045688389073983635
weight_L2 26.936752397445836
tf_result: -0
tf_result: -0
tf_result: 0.045688389073983635
weight_L2 27.712190791263296
tf_result: -0
tf_result: 0.25444653832353925
weight_L2 11.253033764993079
tf_result: 0.68874618927002773
weight_L2 6.905164235557363
tf_result: -0
tf_result: 0.2120925

tf_result: 0.18920898053816695
weight_L2 13.568150255772483
tf_result: 0.045688389073983635
weight_L2 26.691403109520941
tf_result: 0.29303652663743329
weight_L2 10.636911140587245
tf_result: -0
tf_result: 0.045688389073983635
weight_L2 27.083222759356882
tf_result: -0
tf_result: 0.045688389073983635
weight_L2 26.972841673835411
tf_result: -0
tf_result: -0
tf_result: 0.59118916566040192
weight_L2 6.9240065889490143
tf_result: 0.69949182575760893
weight_L2 6.5726898093325419
tf_result: 0.53855694626094208
weight_L2 7.42783620947935
tf_result: 0.75881809502060316
weight_L2 6.0174664061831313
tf_result: 0.65423612345193494
weight_L2 6.688152216462119
tf_result: 1.168185412342851
weight_L2 3.4549460348488017
tf_result: 1.2667268675482317
weight_L2 4.1513700596257888
tf_result: 1.1316696756114457
weight_L2 4.498976797342161
tf_result: 1.279011253188667
weight_L2 4.202940180864136
tf_result: 1.1944518953927323
weight_L2 4.6136829324886115
tf_result: 0.57405354180349255
weight_L2 7.3272433478

weight_L2 27.417728915828967
tf_result: 0.3612835722709638
weight_L2 9.806250691195741
tf_result: -0
tf_result: -0
tf_result: -0
tf_result: 0.045688389073983635
weight_L2 27.4608449276344
tf_result: -0
tf_result: 0.045688389073983635
weight_L2 28.235742504103321
tf_result: 0.045688389073983635
weight_L2 26.729079702168367
tf_result: 0.13906079845474112
weight_L2 16.071159693901411
tf_result: 0.045688389073983635
weight_L2 26.963149431813971
tf_result: 0.16495319147447771
weight_L2 13.675313682078563
tf_result: 0.080484837068006854
weight_L2 20.848322061664888
tf_result: 0.045688389073983635
weight_L2 27.761533186637276
tf_result: 0.11113140431967991
weight_L2 16.77216022634552
tf_result: -0
tf_result: 0.045688389073983635
weight_L2 27.600494946146089
tf_result: -0
tf_result: 0.62693104354049667
weight_L2 6.5438757216690835
tf_result: 0.67258169085788677
weight_L2 6.7425789240388916
tf_result: 0.57727030530027679
weight_L2 7.2405920138702431
tf_result: 0.76251603945596225
weight_L2 6.22

tf_result: 0.45946556602618455
weight_L2 7.8766329280048
tf_result: 0.37677016125643675
weight_L2 9.2641189267554953
tf_result: 0.13906079845474112
weight_L2 15.042818026631315
tf_result: 0.34520662539160074
weight_L2 9.9877899356006843
tf_result: -0
tf_result: 0.080484837068006854
weight_L2 20.20625352316415
tf_result: -0
tf_result: -0
tf_result: -0
tf_result: -0
tf_result: 0.16495319147447771
weight_L2 13.507480968802486
tf_result: 0.65744183572555248
weight_L2 7.1108658411283159
tf_result: 0.045688389073983635
weight_L2 26.34149777202845
tf_result: 0.2120925525087354
weight_L2 12.046002048038938
tf_result: -0
tf_result: 0.37677016125643675
weight_L2 9.1767545496232028
tf_result: 0.045688389073983635
weight_L2 28.114341460482326
tf_result: 0.045688389073983635
weight_L2 26.638681171022924
tf_result: 0.045688389073983635
weight_L2 27.477229038805245
tf_result: 0.045688389073983635
weight_L2 27.754400264887575
tf_result: -0
tf_result: 0.045688389073983635
weight_L2 27.276280763874897
t

weight_L2 6.3119775034117218
tf_result: 0.6019239717322894
weight_L2 7.1138987605799038
tf_result: 0.76251603945596225
weight_L2 6.0150339727854307
tf_result: 0.70643474076717383
weight_L2 6.5426195297007634
tf_result: 1.0828579459025096
weight_L2 3.7970999909213119
tf_result: 1.2534372022259364
weight_L2 4.5297080461435968
tf_result: 1.2827637922754402
weight_L2 4.2067681174694629
tf_result: 1.2683010406516129
weight_L2 4.277765153839292
tf_result: 1.2753466941882163
weight_L2 4.4262691788796182
tf_result: 0.75555352663389663
weight_L2 6.1103697210845551
tf_result: 0.78563526524915239
weight_L2 4.8223075943666824
tf_result: 0.93974411862163232
weight_L2 5.6022718295449234
tf_result: 1.0512605974245017
weight_L2 4.3740304874281106
tf_result: 0.48167812080245515
weight_L2 7.6649617384233455
tf_result: 0.31112915481184922
weight_L2 10.252204768767484
tf_result: -0
tf_result: 0.39169839521161126
weight_L2 9.6245518141348381
tf_result: 0.045688389073983635
weight_L2 27.026326965270009
tf_r

...

tf_result: 0.045688389073983635
weight_L2 26.252951747416727
tf_result: 0.18920898053816695
weight_L2 13.547042254175153
tf_result: -0
tf_result: 0.32850263572607236
weight_L2 9.408282577508329
tf_result: 0.045688389073983635
weight_L2 27.004323390091962
tf_result: 0.045688389073983635
weight_L2 25.704108677669542
tf_result: 0.080484837068006854
weight_L2 18.993558275989439
tf_result: -0
tf_result: -0
tf_result: -0
tf_result: 0.57970811997958638
weight_L2 7.1900842831340075
tf_result: 0.7731496325577546
weight_L2 6.24279401268744
tf_result: 0.55536964463944039
weight_L2 7.2836454664276618
tf_result: 0.718295902951187
weight_L2 6.0879169345881214
tf_result: 0.68699297315178565
weight_L2 6.6053118603015024
tf_result: 1.0733929933219615
weight_L2 3.71666224101282
tf_result: 1.2313706575253625
weight_L2 4.35629392935775
tf_result: 1.2808451608446809
weight_L2 4.0422748406928255
tf_result: 1.167756043239262
weight_L2 4.4637740923448588
tf_result: 1.2049978880233014
weight_L2 4.6064678088797

weight_L2 9.7481353817401128
tf_result: -0
tf_result: 0.045688389073983635
weight_L2 26.85115735245051
tf_result: -0
tf_result: -0
tf_result: -0
tf_result: -0
tf_result: 0.2120925525087354
weight_L2 12.038483274882628
tf_result: 0.64840852574163277
weight_L2 7.1475735393066619
tf_result: 0.045688389073983635
weight_L2 25.464935776161827
tf_result: 0.25444653832353925
weight_L2 11.11638257414891
tf_result: -0
tf_result: 0.37677016125643675
weight_L2 9.0261483534655209
tf_result: 0.080484837068006854
weight_L2 19.514712765505102
tf_result: 0.11113140431967991
weight_L2 15.810770385373649
tf_result: -0
tf_result: -0
tf_result: -0
tf_result: 0.080484837068006854
weight_L2 20.3735593341364
tf_result: -0
tf_result: 0.16495319147447771
weight_L2 14.185033841704396
tf_result: -0
tf_result: 0.41998917198884345
weight_L2 8.3273514462025027
tf_result: 0.11113140431967991
weight_L2 17.290655762209237
tf_result: -0
tf_result: 0.045688389073983635
weight_L2 24.703753961648811
tf_result: -0
tf_result

tf_result: 0.51256293872160963
weight_L2 7.5258026366903579
Time for epoch 16 is 0.9892654418945312 sec - gen_loss = 18.9162741574467, disc_loss = 0.0024044095400143745
tf_result: 0.31112915481184922
weight_L2 9.862883928082752
tf_result: 0.080484837068006854
weight_L2 18.855289068687021
tf_result: 0.39169839521161126
weight_L2 9.4515384279223511
tf_result: -0
tf_result: -0
tf_result: -0
tf_result: -0
tf_result: 0.045688389073983635
weight_L2 25.860346252581643
tf_result: -0
tf_result: 0.080484837068006854
weight_L2 19.483428077329261
tf_result: 0.65744183572555248
weight_L2 7.0036858483096305
tf_result: 0.045688389073983635
weight_L2 26.145300380550641
tf_result: 0.16495319147447771
weight_L2 13.562231781315022
tf_result: -0
tf_result: 0.2120925525087354
weight_L2 12.15065849587139
tf_result: 0.045688389073983635
weight_L2 26.044925534509478
tf_result: 0.045688389073983635
weight_L2 24.406325815560916
tf_result: -0
tf_result: 0.045688389073983635
weight_L2 26.485346909181139
tf_result

weight_L2 6.0007158921819181
tf_result: 0.55987882471534767
weight_L2 7.2077020170614734
tf_result: 0.80072176850501686
weight_L2 5.6774774401415948
tf_result: 0.68975463180221452
weight_L2 6.5652419641155131
tf_result: 1.1109833349478446
weight_L2 3.7990938337486462
tf_result: 1.2233458842454992
weight_L2 4.4879324111592
tf_result: 1.2191019616720302
weight_L2 4.14995891935294
tf_result: 1.3377539863247165
weight_L2 3.9369010565891238
tf_result: 1.2113560403168881
weight_L2 4.4743808979649016
tf_result: 0.59393628467367665
weight_L2 6.6811694628381844
tf_result: 0.86005083881317268
weight_L2 4.7516335408159227
tf_result: 0.89925559535705846
weight_L2 5.4700444397859762
tf_result: 0.96924794541900028
weight_L2 3.9458138514707848
tf_result: 0.6359116020802873
weight_L2 6.5229030275983844


KeyboardInterrupt: 

In [ ]:
arr=np.array([])
noise = tf.random.normal([128, latent_dim])
count=0
for data, labels in training_dataset:
    gen_=generator(noise, data).numpy()
#     arr=np.append(arr,gen_,)
#     if count==2:
#         break
#     count+=1

In [ ]:
# gen_=arr.reshape(-1,62)

In [ ]:
output_dataset = pd.DataFrame(np.round(sc.inverse_transform(gen_)), columns=[
    'BT_NM', 'HR_NM', 'RR_NM', 'HB_NM', 'HCT_NM', 'PLATELET_NM', 'WBC_NM',
    'PTT1_NM', 'PTT2_NM', 'PTINR_NM', 'ER_NM', 'BUN_NM', 'CRE_NM', 'BMI',
    'age', 'PPD', 'THDA_FL', 'THDH_FL', 'THDI_FL', 'THDAM_FL', 'THDV_FL',
    'THDE_FL', 'THDM_FL', 'THDR_FL', 'THDP_FL', 'THDOO_FL', 'Gender',
    'cortical_ACA_ctr', 'cortical_MCA_ctr', 'subcortical_ACA_ctr',
    'subcortical_MCA_ctr', 'PCA_cortex_ctr', 'thalamus_ctr',
    'brainstem_ctr', 'cerebellum_ctr', 'Watershed_ctr',
    'Hemorrhagic_infarct_ctr', 'cortical_ACA_ctl', 'cortical_MCA_ctl',
    'subcortical_ACA_ctl', 'subcortical_MCA_ctl', 'PCA_cortex_ctl',
    'thalamus_ctl', 'brainstem_ctl', 'cerebellum_ctl', 'Watershed_ctl',
    'Hemorrhagic_infarct_ctl', 'NIHS_1a_in', 'NIHS_1b_in', 'NIHS_1c_in',
    'NIHS_2_in', 'NIHS_3_in', 'NIHS_4_in', 'NIHS_5aL_in', 'NIHS_5bR_in',
    'NIHS_6aL_in', 'NIHS_6bR_in', 'NIHS_7_in', 'NIHS_8_in', 'NIHS_9_in',
    'NIHS_10_in', 'NIHS_11_in',
])
output_dataset

In [ ]:
# output_dataset.to_csv("../dataset/output_dataset/cWGAN_1d_2_models.csv",encoding='utf_8_sig')

In [ ]:
output_dataset = pd.DataFrame(np.round(sc.inverse_transform(data)), columns=[
    'BT_NM', 'HR_NM', 'RR_NM', 'HB_NM', 'HCT_NM', 'PLATELET_NM', 'WBC_NM',
    'PTT1_NM', 'PTT2_NM', 'PTINR_NM', 'ER_NM', 'BUN_NM', 'CRE_NM', 'BMI',
    'age', 'PPD', 'THDA_FL', 'THDH_FL', 'THDI_FL', 'THDAM_FL', 'THDV_FL',
    'THDE_FL', 'THDM_FL', 'THDR_FL', 'THDP_FL', 'THDOO_FL', 'Gender',
    'cortical_ACA_ctr', 'cortical_MCA_ctr', 'subcortical_ACA_ctr',
    'subcortical_MCA_ctr', 'PCA_cortex_ctr', 'thalamus_ctr',
    'brainstem_ctr', 'cerebellum_ctr', 'Watershed_ctr',
    'Hemorrhagic_infarct_ctr', 'cortical_ACA_ctl', 'cortical_MCA_ctl',
    'subcortical_ACA_ctl', 'subcortical_MCA_ctl', 'PCA_cortex_ctl',
    'thalamus_ctl', 'brainstem_ctl', 'cerebellum_ctl', 'Watershed_ctl',
    'Hemorrhagic_infarct_ctl', 'NIHS_1a_in', 'NIHS_1b_in', 'NIHS_1c_in',
    'NIHS_2_in', 'NIHS_3_in', 'NIHS_4_in', 'NIHS_5aL_in', 'NIHS_5bR_in',
    'NIHS_6aL_in', 'NIHS_6bR_in', 'NIHS_7_in', 'NIHS_8_in', 'NIHS_9_in',
    'NIHS_10_in', 'NIHS_11_in',
])
output_dataset